<a href="https://colab.research.google.com/github/HunterAshby/DS19-Unit-1-Build/blob/master/Unit_1_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px

In [2]:
df1= pd.read_csv('https://raw.githubusercontent.com/HunterAshby/DS19-Unit-1-Build/master/Covid-19%20Datasets/country_wise_latest.csv')
print(df1.shape)
df1.head()

(187, 15)


,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [3]:
df2 = pd.read_csv('https://raw.githubusercontent.com/HunterAshby/DS19-Unit-1-Build/master/Covid-19%20Datasets/worldometer_data.csv')
df2 = df2[['Country/Region', 'Continent', 'Population', 'TotalTests']]
df2.replace(to_replace={'USA': 'US', 'S. Korea': 'South Korea'}, inplace=True)
print(df2.shape)
df2.head(10)

(209, 4)


,Country/Region,Continent,Population,TotalTests
0,US,North America,3.311981e+08,63139605.0
1,Brazil,South America,2.127107e+08,13206188.0
2,India,Asia,1.381345e+09,22149351.0
3,Russia,Europe,1.459409e+08,29716907.0
4,South Africa,Africa,5.938157e+07,3149807.0
5,Mexico,North America,1.290662e+08,1056915.0
6,Peru,South America,3.301632e+07,2493429.0
7,Chile,South America,1.913251e+07,1760615.0
8,Colombia,South America,5.093626e+07,1801835.0
9,Spain,Europe,4.675665e+07,7064329.0


In [4]:
df = df1.merge(df2, on='Country/Region', how=('left'))
print(df.shape)
df.loc[177, 'Population'] = 66650000
df.head()

(187, 18)


,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region,Continent,Population,TotalTests
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean,Asia,39009447.0,90396.0
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe,Europe,2877470.0,38997.0
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa,Africa,43926079.0,NaN
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe,Europe,77278.0,3750.0
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa,Africa,32956300.0,64747.0


In [5]:
df_worldwide = pd.read_csv('https://raw.githubusercontent.com/HunterAshby/DS19-Unit-1-Build/master/Covid-19%20Datasets/day_wise.csv')
print(df_worldwide.shape)
df_worldwide['Date'] = pd.to_datetime(df_worldwide['Date'])
df_worldwide['New cases'] = df_worldwide['New cases'].astype('float64')
df_worldwide.head()

(188, 12)


,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0.0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99.0,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287.0,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493.0,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684.0,14,13,2.64,2.46,107.69,13


In [6]:
df_day = pd.read_csv('https://raw.githubusercontent.com/HunterAshby/DS19-Unit-1-Build/master/Covid-19%20Datasets/full_grouped.csv')
df_day['Date'] = pd.to_datetime(df_day['Date'])
df_day['New cases'] = df_day['New cases'].astype('float64')
df_day.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0.0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0.0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0.0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0.0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0.0,0,0,Africa


In [7]:
#Matplotlib graph maker
#Made a plotly one, but leaving this one here
def compare_countries(countries, how='New Cases'):
  months = mdates.MonthLocator()
  months_fmt = mdates.DateFormatter('%M')
  fig, ax = plt.subplots()
  title = ''
  world_pop = df['Population'].sum()
  
  if type(countries) == str:
    if countries == 'World':
      ax.plot(df_worldwide['Date'], (df_worldwide['New cases']/world_pop)*100, label='World')
    else:
      country_pop = df[df['Country/Region'] == countries]['Population']
      dfi = df_day[df_day['Country/Region'] == countries]
      ax.plot(dfi['Date'], (dfi['New cases']/country_pop.iloc[0])*100, label=i)
    ax.set_title(countries + ' Covid Cases')
    plt.legend(loc='upper left')
  else:
    for i in countries:
      title = title + (i + ' VS ') 
      if i == 'World':
        ax.plot(df_worldwide['Date'], (df_worldwide['New cases']/world_pop)*100, label='World')
      else:
        country_pop = df[df['Country/Region'] == i]['Population']
        dfi = df_day[df_day['Country/Region'] == i]
        ax.plot(dfi['Date'], (dfi['New cases']/country_pop.iloc[0])*100, label=i)
    if len(countries) <= 5:
      ax.set_title(title.rstrip('VS ') + ' Covid-19 Cases')
      plt.legend(loc='upper left')
    else:
      ax.set_title('Covid-19 Cases')

  ax.set_xlabel('Date')
  ax.set_ylabel('New Cases %of population')
  ax.xaxis.set_major_locator(months)
  ax.xaxis.set_major_formatter(months_fmt)
  ax.set_xticklabels(['Feburary', 'March', 'April', 'May', 'June', 'July', 'August'])
  plt.show()

In [8]:
#Plotly graph maker
def compare_countries(countries, on='New cases', how='%'):
  #Setting up needed values
  pd.set_option('mode.chained_assignment', None)
  plt_title = ''
  dfi = pd.DataFrame()
  world_pop = df['Population'].sum()
  
  #If the person only passes in one values
  if type(countries) == str:
    #If the person wants to see the value of the worldwide crisis
    if countries == 'World':
      #Runs if user wants the data to be a percentage of the population 
      #total for comparability, by defult this is true
      if how == '%':
        fig = px.line(df_worldwide, x='Date', y=(df_worldwide[on]/world_pop)*100,
                      title='Worldwide Covid Cases', labels={'y': on + ' % of Pop'}, 
                      hover_data=[on])
      else:
        #Else the person will see total values
        fig = px.line(df_worldwide, x='Date', y=df_worldwide[on],
                      title='Worldwide Covid Cases', labels={'y': on}, 
                      hover_data=[on])
    else:
      #Grabs population total for given country
      country_pop = df[df['Country/Region'] == countries]['Population']
      dfi = df_day[df_day['Country/Region'] == countries]
      #Percentage of Population Numbers
      if how == '%':
        fig = px.line(dfi, x='Date', y=(dfi[on]/country_pop.iloc[0])*100,
                      title=countries + ' Covid Cases', labels={'y': on + ' % of Pop'}, 
                      hover_data=[on])
      else:
        #Total numbers
        fig = px.line(dfi, x='Date', y=dfi[on],
                      title=countries + ' Covid Cases', labels={'y': on}, 
                      hover_data=[on])
  else:
    #Builds a new DF called dfi that holds all the info need to graph the specified countries
    for i in countries:
      #Builds the Title of the graph
      plt_title = plt_title + (i + ' VS ') 
      if i == 'World':
        #Worldwide Values
        dfiw = df_worldwide
        dfiw['% of Pop'] = dfiw[on]/world_pop
        dfi = dfi.append(dfiw)
        dfi.sort_values('Date', axis=0, ascending=False, inplace=True)

      else:
        #Building DF with Specified Countries
        dfi1 = df_day[df_day['Country/Region'] == i]
        country_pop = df[df['Country/Region'] == i]['Population']
        dfi1['% of Pop'] = (dfi1[on]/country_pop.iloc[0])
        dfi = dfi.append(dfi1)
        dfi.sort_values('Date', axis=0, ascending=False, inplace=True)

    #Will only display title and legend if the total countries number 
    #less than or equal to 10
    if len(countries) <= 10:
      #Percentage of Population Numbers
      if how == '%':
        fig = px.line(dfi, x='Date', y=dfi['% of Pop']*100, color='Country/Region', 
                      title=plt_title.rstrip(' VS ')  + ' Covid Cases', 
                      labels={'Country/Region': '', 'y' : on +' % of Pop'}, 
                      hover_data=[on])
      #Total Numbers
      else:
        fig = px.line(dfi, x='Date', y=dfi[on], color='Country/Region', 
                      title=plt_title.rstrip(' VS ')  + ' Covid Cases', 
                      labels={'Country/Region': '', 'y' : on}, 
                      hover_data=[on])
    #Builds Graph without legend and Title
    else:
      #Percentage of Population Numbers
      if how == '%':
        fig = px.line(dfi, x='Date', y=dfi['% of Pop']*100, 
                      color='Country/Region', labels={'y': on + ' % of Pop'}, 
                      hover_data=[on])
      #Total Numbers
      else:
        fig = px.line(dfi, x='Date', y=dfi[on], 
                      color='Country/Region', labels={'y': on}, 
                      hover_data=[on])
      fig.update_layout(showlegend=False)

  fig.show()
  pd.set_option('mode.chained_assignment', 'warn')

In [9]:
compare_countries(['Afghanistan', 'Albania'], on='Confirmed')

In [10]:
compare_countries(df['Country/Region'], 'Active', how='%')

In [11]:
compare_countries(['US', 'Sweden'], 'Confirmed', '%')

In [12]:
compare_countries('World', on='Confirmed', how='%')

In [13]:
df.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region,Continent,Population,TotalTests
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean,Asia,39009447.0,90396.0
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe,Europe,2877470.0,38997.0
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa,Africa,43926079.0,NaN
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe,Europe,77278.0,3750.0
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa,Africa,32956300.0,64747.0


In [14]:
def compare_countries_totals(countries, on='Confirmed', how='%'):
  #Will not plot 1 country, or World values
  #Setting up necessary values
  pd.set_option('mode.chained_assignment', None)
  plt_title = ''
  dfi = pd.DataFrame()
  world_pop = df['Population'].sum()

  #Iterates through list of given countries to grab relevent data for graphing
  for i in countries:
    plt_title = plt_title + (i + ' VS ')
    dfi1 = df[df['Country/Region'] == i]
    dfi1['% of Pop'] = dfi1[on]/dfi1['Population']
    dfi = dfi.append(dfi1)

  #Runs if user wants the data to be a percentage of the population 
  #total for comparability, by defult this is true
  if how == '%':
    fig = px.bar(dfi, x='Country/Region', y=dfi['% of Pop']*100, color='Country/Region', 
                 title=plt_title.rstrip(' VS ')  + ' Covid Cases', 
                 labels={'Country/Region': '', 'y' : on +' % of Pop'}, 
                 hover_data=[on, 'Population'])
                 
    fig.update_layout(showlegend=False)

  #Runs if user just wants the total numbers to compare
  else:
    fig = px.bar(dfi, x='Country/Region', y=on, color='Country/Region',
                 title=plt_title.rstrip(' VS ')  + ' Covid Cases', 
                 labels={'Country/Region': '', 'y' : on}, 
                 hover_data=[on, 'Population'])
    
    fig.update_layout(showlegend=False)

  fig.show()
  pd.set_option('mode.chained_assignment', 'warn')

In [15]:
compare_countries_totals(['Afghanistan', 'US'], on='Deaths')

In [16]:
compare_countries_totals(df['Country/Region'], on='Active', how="Total")

In [17]:
compare_countries_totals(['US', 'Mexico', 'Germany', 'United Kingdom', 'Canada', 'Italy'], on='Confirmed', how='%')

In [18]:
compare_countries(['US', 'Mexico', 'Germany', 'United Kingdom', 'Canada', 'Italy'], on='Active', how='%')